## Recommendation System

In [1]:
import firebase_admin
from firebase_admin import db,credentials
import main as recommender
import pandas as pd
from random import sample
cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred, {'databaseURL': 'https://fir-demo-29d5b-default-rtdb.firebaseio.com'})

### Get data from Firebase Realtime DB

In [2]:
ref = db.reference('/')
data = ref.child('songs').order_by_child('id').get()
songs = []
for key in data:
    songs.append(data[key])

In [3]:
songs_df = pd.DataFrame(songs).dropna().drop_duplicates(['id'])

song_stat_df = songs_df.copy()
song_stat_df.drop(columns=['320kbps','album_id' ,'duration','has_lyrics' , 'image','perma_url', 'url', 'release_date', 'genre' ,'album','language' , 'subtitle','year'],inplace=True) #drop all of unnecessary columns 
# to convert string into list of strings 
def spliting(text):
  text=str(text).split()
  return text
song_stat_df['title'] = song_stat_df['title'].apply(spliting)
song_stat_df['artist'] = song_stat_df['artist'].apply(spliting)
song_stat_df['all_tags'] = song_stat_df['title'] + song_stat_df['artist'] 
song_stat_df.drop(columns=['title','artist'],inplace=True)
song_stat_df

,id,all_tags
0,-4eFAzI-7o0,"[THƯƠNG, THÌ, VỀ, ĐÂY, |, TIẾN, TỚI, |, Offici..."
1,-5q5mZbe3V8,"[BTS, (방탄소년단), 'Life, Goes, On', Official, MV,..."
2,-7sISWuTdj0,"[PHÚC, DU, -, yêu, anh, đi, mẹ, anh, bán, bánh..."
3,-E2u6exhzPE,"[MAKING, MY, WAY, Sơn, Tùng, M-TP]"
4,-GQg25oP0S4,"[SEVENTEEN, (세븐틴), '손오공', Official, MV, HYBE, ..."
...,...,...
1512,zlTIextYnyQ,"[KAI, 카이, 'Rover', MV, SMTOWN]"
1513,znlFu_lemsU,"[Mary, J., Blige, -, Family, Affair, (Official..."
1514,znvky0Uq8qc,"[Until, I, Found, You, Stephen, Sanchez]"
1515,zoC9Lxt2ZJU,"[Đừng, Chúc, Em, Hạnh, Phúc, (Lyrics, Video), ..."


In [4]:
def convert_lower(text):
  l=[]
  for item in text:
    l.append(item.lower())
  return l
song_stat_df['all_tags']=song_stat_df['all_tags'].apply(convert_lower)
song_stat_df

,id,all_tags
0,-4eFAzI-7o0,"[thương, thì, về, đây, |, tiến, tới, |, offici..."
1,-5q5mZbe3V8,"[bts, (방탄소년단), 'life, goes, on', official, mv,..."
2,-7sISWuTdj0,"[phúc, du, -, yêu, anh, đi, mẹ, anh, bán, bánh..."
3,-E2u6exhzPE,"[making, my, way, sơn, tùng, m-tp]"
4,-GQg25oP0S4,"[seventeen, (세븐틴), '손오공', official, mv, hybe, ..."
...,...,...
1512,zlTIextYnyQ,"[kai, 카이, 'rover', mv, smtown]"
1513,znlFu_lemsU,"[mary, j., blige, -, family, affair, (official..."
1514,znvky0Uq8qc,"[until, i, found, you, stephen, sanchez]"
1515,zoC9Lxt2ZJU,"[đừng, chúc, em, hạnh, phúc, (lyrics, video), ..."


In [5]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def steming(text):
  l=[]
  for i in text:
    l.append(ps.stem(i))
  return l

song_stat_df['all_tags'] = song_stat_df['all_tags'].apply(steming)
song_stat_df['all_tags'] = song_stat_df['all_tags'].apply(lambda x: " ".join(x))
song_stat_df

,id,all_tags
0,-4eFAzI-7o0,thương thì về đâi | tiến tới | offici music vi...
1,-5q5mZbe3V8,bt (방탄소년단) 'life goe on' offici mv hybe label
2,-7sISWuTdj0,phúc du - yêu anh đi mẹ anh bán bánh mì (mv of...
3,-E2u6exhzPE,make my way sơn tùng m-tp
4,-GQg25oP0S4,seventeen (세븐틴) '손오공' offici mv hybe label
...,...,...
1512,zlTIextYnyQ,kai 카이 'rover' mv smtown
1513,znlFu_lemsU,mari j. blige - famili affair (offici music vi...
1514,znvky0Uq8qc,until i found you stephen sanchez
1515,zoC9Lxt2ZJU,đừng chúc em hạnh phúc (lyric video) - thanh h...


In [6]:
keywork = ['Changg',
 'dhruv',
 'Da LAB']
work = ' '
work = work.join(keywork)
work.lower()

'changg dhruv da lab'

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
all_tags = song_stat_df['all_tags']
all_tags[len(all_tags)] = work
data = vectorizer.fit_transform(all_tags)
print("Unique vocabulary: ", len(vectorizer.vocabulary_))
data.toarray()

Unique vocabulary:  3469


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(data)
similarity

array([[1.        , 0.01579025, 0.        , ..., 0.        , 0.00744965,
        0.        ],
       [0.01579025, 1.        , 0.0248851 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.0248851 , 1.        , ..., 0.        , 0.08244809,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.00744965, 0.        , 0.08244809, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [19]:
def recommend(song):
    index = song_stat_df[song_stat_df['id'] == song].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    list_id = []
    for i in distances:
        if len(list_id) > 20:
            break
        id = song_stat_df.iloc[i[0]].id
        if id not in list_id:
            list_id.append(id)
    columns = ['320kbps','album','album_id','artist','duration','genre','has_lyrics','id','image','language','perma_url','release_date','subtitle','title','url','year']
    list_result = pd.DataFrame(columns= columns)
    for id in list_id:
        result = songs_df.query('id == @id')
        list_result = pd.concat([list_result, result], ignore_index=True).drop_duplicates(['id'])
    return list_result
    
recommend("jO2viLEW-1A")

,320kbps,album,album_id,artist,duration,genre,has_lyrics,id,image,language,perma_url,release_date,subtitle,title,url,year
0,false,Twenty One Pilots,UCnX0L9QiftAcWdzeBx31xCw,Twenty One Pilots,196,YouTube,false,oLeROuCMwj8,https://img.youtube.com/vi/oLeROuCMwj8/maxresd...,YouTube,https://www.youtube.com/watch?v=oLeROuCMwj8,2017-02-09 00:00:00.000,Twenty One Pilots - Topic,Heathens,https://rr2---sn-8qj-nbo6d.googlevideo.com/vid...,2017
1,false,twenty one pilots,UCBQZwaNPFfJ1gZ1fLZpAEGw,twenty one pilots,225,YouTube,false,Pw-0pbY9JeU,https://img.youtube.com/vi/Pw-0pbY9JeU/maxresd...,YouTube,https://www.youtube.com/watch?v=Pw-0pbY9JeU,2015-05-13 00:00:00.000,twenty one pilots,twenty one pilots - Ride (Official Video),https://rr4---sn-42u-nbozl.googlevideo.com/vid...,2015
2,false,Fueled By Ramen,UClVrJwcIy7saPcGc1nct80A,Fueled By Ramen,218,YouTube,false,UprcpdwuwCg,https://img.youtube.com/vi/UprcpdwuwCg/maxresd...,YouTube,https://www.youtube.com/watch?v=UprcpdwuwCg,2016-06-21 00:00:00.000,Fueled By Ramen,twenty one pilots: Heathens (from Suicide Squa...,https://rr1---sn-8qj-nbo6d.googlevideo.com/vid...,2016
3,false,Fueled By Ramen,UClVrJwcIy7saPcGc1nct80A,Fueled By Ramen,225,YouTube,false,pXRviuL6vMY,https://img.youtube.com/vi/pXRviuL6vMY/maxresd...,YouTube,https://www.youtube.com/watch?v=pXRviuL6vMY,2015-04-27 00:00:00.000,Fueled By Ramen,twenty one pilots: Stressed Out [OFFICIAL VIDEO],https://rr2---sn-8qj-nbo6d.googlevideo.com/vid...,2015
4,false,Calvin Harris,UCZ0Aezmtk-S2l8A9Ln-2lKw,Calvin Harris,215,YouTube,false,XWCPbRsMb0Q,https://img.youtube.com/vi/XWCPbRsMb0Q/maxresd...,YouTube,https://www.youtube.com/watch?v=XWCPbRsMb0Q,2018-04-05 00:00:00.000,Calvin Harris - Topic,One Kiss,https://rr8---sn-8qj-nboey.googlevideo.com/vid...,2018
5,false,Number One Hits,UCaVGVfuM8qWgyORw14-91sA,Number One Hits,280,YouTube,false,0UMAWfyxs4c,https://img.youtube.com/vi/0UMAWfyxs4c/maxresd...,YouTube,https://www.youtube.com/watch?v=0UMAWfyxs4c,2017-09-14 00:00:00.000,Number One Hits - Topic,Despacito,https://rr2---sn-8qj-nbo6d.googlevideo.com/vid...,2017
6,false,Ariana Grande,UC0076UMUgEng8HORUw_MYHA,Ariana Grande,197,YouTube,false,KEGZVv6KXL0,https://img.youtube.com/vi/KEGZVv6KXL0/maxresd...,YouTube,https://www.youtube.com/watch?v=KEGZVv6KXL0,2018-07-23 00:00:00.000,Ariana Grande - Topic,One Last Time,https://rr2---sn-8qj-nbo6d.googlevideo.com/vid...,2018
7,false,One Direction,UCREhun2tPd7aQnJczZM_Mww,One Direction,200,YouTube,false,d57qSDCxVA4,https://img.youtube.com/vi/d57qSDCxVA4/maxresd...,YouTube,https://www.youtube.com/watch?v=d57qSDCxVA4,2017-02-18 00:00:00.000,One Direction - Topic,What Makes You Beautiful,https://rr7---sn-8qj-nboey.googlevideo.com/vid...,2017
8,false,Maroon5VEVO,UCN1hnUccO4FD5WfM7ithXaw,Maroon5VEVO,223,YouTube,false,fwK7ggA3-bU,https://img.youtube.com/vi/fwK7ggA3-bU/maxresd...,YouTube,https://www.youtube.com/watch?v=fwK7ggA3-bU,2012-06-26 00:00:00.000,Maroon5VEVO,Maroon 5 - One More Night (Official Music Video),https://rr7---sn-42u-nbozz.googlevideo.com/vid...,2012
9,false,Teddy Swims,UCAtlzufZmPhkiDApWM4bTRQ,Teddy Swims,196,YouTube,false,jEylXhpABMs,https://img.youtube.com/vi/jEylXhpABMs/maxresd...,YouTube,https://www.youtube.com/watch?v=jEylXhpABMs,2020-07-16 00:00:00.000,Teddy Swims - Topic,You're Still The One,https://rr1---sn-8qj-nbo6d.googlevideo.com/vid...,2020


## API

In [ ]:
import service as sv
self_sv = sv.firebase_service()
self_sv.get_data_and_preprocessing()

In [ ]:
self_sv.random_update_stats()